In [1]:
import sys
sys.path.append('/Users/spindicate/Documents/programming/zooscraper/')

In [2]:
import django
django.setup()

In [3]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [4]:
from datetime import datetime as dt, timedelta
import numpy as np
import pandas as pd

import xarray as xr

from zooscraper.globals import max_bulk_create
from casestudy.models import Measurements

In [39]:
def convert_long(long):
    if long < 0:
        long = 180 - long
    return long

In [40]:
# def update_msmts(save=False):
msmt_path = '/Users/spindicate/Documents/docs/covid19/msmt/'
msmt_files = [file for file in os.listdir(msmt_path) if '.nc' in file]

temp_sets = []
uvb_sets = []
for msmt_file in msmt_files:
    dataset = xr.open_dataset(msmt_path + msmt_file)
    if 'expver' in dataset.variables:
        dataset = dataset.sel(expver=5)
        del dataset['expver']
    if 't2m' in dataset.variables:
        temp_sets.append(dataset)
    elif 'uvb' in dataset.variables:
        uvb_sets.append(dataset)

In [41]:
uvb_sets

[<xarray.Dataset>
 Dimensions:    (latitude: 721, longitude: 1440, time: 384)
 Coordinates:
   * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.25 359.5 359.75
   * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
   * time       (time) datetime64[ns] 2020-03-01 ... 2020-03-16T23:00:00
 Data variables:
     uvb        (time, latitude, longitude) float32 ...
 Attributes:
     Conventions:  CF-1.6
     history:      2020-04-19 21:33:37 GMT by grib_to_netcdf-2.16.0: /opt/ecmw...,
 <xarray.Dataset>
 Dimensions:    (latitude: 721, longitude: 1440, time: 360)
 Coordinates:
   * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.25 359.5 359.75
   * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
   * time       (time) datetime64[ns] 2020-03-17 ... 2020-03-31T23:00:00
 Data variables:
     uvb        (time, latitude, longitude) float32 ...
 Attributes:
     Conventions:  CF-1.6
     history:      2020-04-19 21:34:18 GMT by g

In [42]:
ds_temp = xr.combine_by_coords(temp_sets)
ds_new = xr.Dataset({'d2m': ds_temp.d2m.resample(time='D', skipna=True).mean()})
ds_new['t2m'] = ds_temp.t2m.resample(time='D').mean()

In [43]:
ds_uvb = xr.combine_by_coords(uvb_sets)
ds_uvb = ds_uvb.resample(time='D').sum()
ds_new['uvb'] = ds_uvb.uvb

In [48]:
ds_new.sel(latitude=114.2, longitude=22.3, method='nearest')

<xarray.Dataset>
Dimensions:    (time: 112)
Coordinates:
  * time       (time) datetime64[ns] 2020-01-01 2020-01-02 ... 2020-04-21
    longitude  float32 22.25
    latitude   float32 90.0
Data variables:
    d2m        (time) float32 242.7437 240.68071 244.80963 ... 265.468 265.53458
    t2m        (time) float32 245.89268 244.0459 247.6104 ... 266.449 266.62
    uvb        (time) float32 0.75 0.75 0.75 ... 1442087.4 1373275.2 1552907.9

In [47]:
Region.objects.filter(name='Hong Kong').values()

<QuerySet [{'id': 353, 'name': 'Hong Kong', 'name_alt': None, 'country_key_id': 100, 'country': 'Hong Kong', 'country_alt': 'China, Hong Kong Special Administrative Region', 'longitude': 114.2, 'latitude': 22.3, 'LAND_A_KM': 1113.7433397654595, 'land_density': 6309.601679408495, 'city_dens': 9261.845843418043, 'population': 7027276, 'A85PLUSB': 125138, 'A80PLUSB': 270881, 'A75PLUSB': 475547, 'A70PLUSB': 705363, 'A65PLUSB': 938722, 'A09UNDERB': 488332, 'A14UNDERB': 816921, 'A19UNDERB': 1237917, 'A24UNDERB': 1686816, 'A29UNDERB': 2217424, 'A34UNDERB': 2763248, 'A05_14B': 569899, 'A05_19B': 990895, 'A05_24B': 1439794, 'A05_29B': 1970402, 'A05_34B': 2516226, 'A15_24B': 869894, 'A15_29B': 1400502, 'A15_34B': 1946326, 'A20_29B': 979507, 'A20_34B': 1525331, 'A35_54B': 2407515, 'A40_54B': 1849236, 'A45_54B': 1280717, 'A35_64B': 3325305, 'A40_64B': 2767026, 'A45_64B': 2198507, 'A80_84B': 145742, 'A75_79B': 204666, 'A70_74B': 229815, 'A65_69B': 233359, 'A60_64B': 407568, 'A55_59B': 510221, 'A50_

In [49]:
msmt_objs = []
for region in Region.objects.filter(longitude__isnull=False):
    print (region.name)
    ds_longlat = ds_new.sel(latitude=region.latitude, longitude=region.longitude, method='nearest')

    times = ds_longlat.t2m.time.values
    uvbs = ds_longlat.uvb.values
    t2ms = ds_longlat.t2m.values
    d2ms = ds_longlat.d2m.values

    for values in zip(list(times), uvbs, t2ms, d2ms):
        date_shift = values[0] + np.timedelta64(12, 'h')
        ts = (date_shift - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's')
        date = dt.utcfromtimestamp(ts)
        print (date, values[1], values[2], values[3])
        msmt_objs.append(
            Measurements(
                region=region, longitude=region.longitude, 
                latitude=region.latitude, date=date,
                uvb=values[1], 
                temp= values[2], dewpoint= values[3]
        ))

P.A. Trento
2020-01-01 12:00:00 633663.6 277.23883 271.69012
2020-01-02 12:00:00 633157.0 278.01166 272.21872
2020-01-03 12:00:00 572544.25 277.31146 272.74265
2020-01-04 12:00:00 575524.6 276.72113 271.54367
2020-01-05 12:00:00 603765.25 275.4285 268.68896
2020-01-06 12:00:00 656381.75 274.44278 270.9221
2020-01-07 12:00:00 635853.9 275.56485 269.78317
2020-01-08 12:00:00 662222.25 275.72665 269.3755
2020-01-09 12:00:00 643274.9 277.79715 270.96677
2020-01-10 12:00:00 612354.4 276.14496 271.3443
2020-01-11 12:00:00 665623.5 276.54794 270.1317
2020-01-12 12:00:00 676943.75 275.66998 270.65942
2020-01-13 12:00:00 689698.5 275.26233 270.1193
2020-01-14 12:00:00 697514.4 273.75827 269.8646
2020-01-15 12:00:00 647474.9 275.04318 271.4271
2020-01-16 12:00:00 677665.25 277.27744 272.33008
2020-01-17 12:00:00 631636.75 277.5236 271.85757
2020-01-18 12:00:00 384968.94 274.76382 273.05356
2020-01-19 12:00:00 746978.6 272.7933 266.0056
2020-01-20 12:00:00 752630.1 272.91412 263.80957
2020-01-21 

/Users/spindicate/Documents/programming/envs/zooenv/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  del sys.path[0]


 905497.5 276.30722 271.59467
2020-01-10 12:00:00 912059.4 276.9888 271.7194
2020-01-11 12:00:00 900438.5 278.09857 273.94302
2020-01-12 12:00:00 657068.75 278.97568 276.18228
2020-01-13 12:00:00 621252.5 278.04468 275.9082
2020-01-14 12:00:00 922821.5 278.59134 274.51797
2020-01-15 12:00:00 965397.25 278.62567 274.30786
2020-01-16 12:00:00 965088.0 277.82697 274.1049
2020-01-17 12:00:00 990880.9 277.29877 271.97955
2020-01-18 12:00:00 691768.4 277.24384 274.71066
2020-01-19 12:00:00 700829.9 278.57254 276.08957
2020-01-20 12:00:00 359339.34 278.10452 276.30014
2020-01-21 12:00:00 843063.9 277.60815 275.27603
2020-01-22 12:00:00 1009931.4 277.2493 273.9971
2020-01-23 12:00:00 1048375.9 278.80453 273.8595
2020-01-24 12:00:00 1025760.9 278.13162 273.5811
2020-01-25 12:00:00 1008643.0 279.5215 275.84274
2020-01-26 12:00:00 843201.25 280.20425 277.65616
2020-01-27 12:00:00 941648.6 280.12732 277.0142
2020-01-28 12:00:00 953707.6 281.98868 278.64297
2020-01-29 12:00:00 975369.1 282.32953 27

2020-01-15 12:00:00 769404.5 266.23068 257.448
2020-01-16 12:00:00 845623.5 264.9429 257.77887
2020-01-17 12:00:00 683686.1 265.13425 259.46402
2020-01-18 12:00:00 813354.6 263.18533 260.99942
2020-01-19 12:00:00 767643.9 260.09454 253.19
2020-01-20 12:00:00 841122.9 259.45877 254.82776
2020-01-21 12:00:00 897441.0 261.81137 256.2983
2020-01-22 12:00:00 920382.25 266.30734 258.20886
2020-01-23 12:00:00 895388.1 266.91452 258.1001
2020-01-24 12:00:00 863316.9 265.8919 260.74295
2020-01-25 12:00:00 883071.6 263.7339 260.91034
2020-01-26 12:00:00 769301.5 262.87476 260.35507
2020-01-27 12:00:00 835248.0 266.10162 263.5601
2020-01-28 12:00:00 572733.25 266.88892 264.50278
2020-01-29 12:00:00 614639.1 267.3937 265.56448
2020-01-30 12:00:00 880666.75 268.242 264.65042
2020-01-31 12:00:00 852975.6 272.2826 270.85068
2020-02-01 12:00:00 778935.75 270.6359 268.5562
2020-02-02 12:00:00 502073.28 272.6907 271.70358
2020-02-03 12:00:00 805292.4 274.94467 272.5618
2020-02-04 12:00:00 751553.0 268.1

2020-01-13 12:00:00 2102230.8 289.5818 267.3798
2020-01-14 12:00:00 2142959.8 290.40622 265.19327
2020-01-15 12:00:00 2139533.0 291.6224 266.00992
2020-01-16 12:00:00 2088815.0 292.90085 267.24768
2020-01-17 12:00:00 2141594.2 293.96613 269.09055
2020-01-18 12:00:00 2166356.2 294.02142 271.42288
2020-01-19 12:00:00 2179824.0 293.67874 270.8588
2020-01-20 12:00:00 2189452.2 294.60883 270.5942
2020-01-21 12:00:00 2195275.5 296.51996 271.0002
2020-01-22 12:00:00 2153069.2 297.87097 270.5904
2020-01-23 12:00:00 2149625.2 299.1973 264.79236
2020-01-24 12:00:00 1555187.9 298.0759 265.12192
2020-01-25 12:00:00 2041618.1 299.1243 263.44797
2020-01-26 12:00:00 2195516.2 298.32175 262.7329
2020-01-27 12:00:00 2088754.8 298.26352 263.25354
2020-01-28 12:00:00 1989276.6 298.72476 259.1669
2020-01-29 12:00:00 2271683.8 298.06866 257.94418
2020-01-30 12:00:00 2261574.2 298.26965 262.08456
2020-01-31 12:00:00 2277232.0 297.79697 265.93903
2020-02-01 12:00:00 2302127.0 297.65585 266.10062
2020-02-02 1

...

2020-03-30 12:00:00 2289625.2 295.36578 291.7082
2020-03-31 12:00:00 2211300.8 295.10538 290.89328
2020-04-01 12:00:00 2497201.5 295.25266 288.96344
2020-04-02 12:00:00 2166661.5 295.14282 289.12802
2020-04-03 12:00:00 2335279.8 294.66556 289.2517
2020-04-04 12:00:00 2346466.8 295.05698 288.54202
2020-04-05 12:00:00 2135944.2 295.41757 289.3241
2020-04-06 12:00:00 2129756.2 295.308 290.6627
2020-04-07 12:00:00 2046627.6 295.20798 290.95554
2020-04-08 12:00:00 1833129.8 295.02905 289.3647
2020-04-09 12:00:00 2025589.2 294.87637 289.52585
2020-04-10 12:00:00 1810993.0 294.61118 288.52637
2020-04-11 12:00:00 2241840.5 295.16052 288.7091
2020-04-12 12:00:00 2091441.5 295.67392 291.014
2020-04-13 12:00:00 2147581.8 295.6825 292.15323
2020-04-14 12:00:00 1639950.4 294.91977 289.5324
2020-04-15 12:00:00 1832017.5 294.9481 288.73685
2020-04-16 12:00:00 1983465.6 295.11472 289.97012
2020-04-17 12:00:00 2231075.2 295.33524 290.3306
2020-04-18 12:00:00 1916075.5 294.41013 288.90076
2020-04-19 12:

2020-01-01 12:00:00 2924217.0 297.93796 294.77206
2020-01-02 12:00:00 2409623.5 297.79916 294.85864
2020-01-03 12:00:00 3032884.8 297.80298 294.44888
2020-01-04 12:00:00 3022707.0 297.7944 293.71896
2020-01-05 12:00:00 3028092.5 297.67657 293.45798
2020-01-06 12:00:00 2931234.0 297.90662 293.35358
2020-01-07 12:00:00 2808729.0 298.00018 293.06726
2020-01-08 12:00:00 3135764.5 298.00964 292.78574
2020-01-09 12:00:00 2886356.8 298.3295 293.6904
2020-01-10 12:00:00 3235431.5 298.42133 294.44058
2020-01-11 12:00:00 3017845.8 298.27548 294.65958
2020-01-12 12:00:00 3083886.8 298.25916 295.22986
2020-01-13 12:00:00 2667104.8 298.283 294.87497
2020-01-14 12:00:00 3061478.0 298.1359 294.43234
2020-01-15 12:00:00 3175033.5 298.5363 293.64252
2020-01-16 12:00:00 3021358.5 298.9185 294.17014
2020-01-17 12:00:00 2879605.5 298.64053 295.55643
2020-01-18 12:00:00 3079789.8 298.75626 295.2162
2020-01-19 12:00:00 2779517.5 298.696 295.1497
2020-01-20 12:00:00 2966767.0 298.8461 294.86957
2020-01-21 12

2020-02-18 12:00:00 2415394.5 300.69763 297.5334
2020-02-19 12:00:00 2821382.2 300.66068 297.35034
2020-02-20 12:00:00 2856606.5 300.83655 297.78873
2020-02-21 12:00:00 1714993.6 300.87424 297.82944
2020-02-22 12:00:00 2857194.0 300.53946 297.6831
2020-02-23 12:00:00 1499176.6 300.99185 298.0107
2020-02-24 12:00:00 2892690.0 300.93985 298.03336
2020-02-25 12:00:00 2919239.5 300.69785 298.25873
2020-02-26 12:00:00 2923238.5 300.5872 297.85397
2020-02-27 12:00:00 3147230.5 300.55713 297.15378
2020-02-28 12:00:00 3120251.2 300.63657 296.3124
2020-02-29 12:00:00 3085992.2 300.5001 296.6238
2020-03-01 12:00:00 3042702.8 300.8402 296.9528
2020-03-02 12:00:00 3005686.8 301.05466 297.5717
2020-03-03 12:00:00 2829934.8 301.01355 298.03128
2020-03-04 12:00:00 3068231.5 300.70438 297.39355
2020-03-05 12:00:00 2954013.8 300.78592 296.03027
2020-03-06 12:00:00 2674866.0 300.85727 296.14978
2020-03-07 12:00:00 2789480.2 300.64023 296.5494
2020-03-08 12:00:00 2948687.0 301.007 295.7615
2020-03-09 12:

2020-03-04 12:00:00 3125332.2 300.8189 297.78098
2020-03-05 12:00:00 2793702.5 300.73917 296.82837
2020-03-06 12:00:00 3059340.0 301.02667 296.53006
2020-03-07 12:00:00 2954013.5 300.9415 296.87988
2020-03-08 12:00:00 2936601.2 301.11444 296.34134
2020-03-09 12:00:00 2853019.5 301.30856 296.14874
2020-03-10 12:00:00 2978404.8 301.41666 296.05527
2020-03-11 12:00:00 2597175.5 301.20248 296.64163
2020-03-12 12:00:00 2894376.2 301.18277 297.59836
2020-03-13 12:00:00 2968830.0 300.95004 297.05045
2020-03-14 12:00:00 2965905.8 301.07425 296.27225
2020-03-15 12:00:00 3164304.2 301.19992 296.42618
2020-03-16 12:00:00 2746623.0 300.7745 297.59686
2020-03-17 12:00:00 2881091.5 301.43512 296.57095
2020-03-18 12:00:00 2978574.2 301.32877 296.04358
2020-03-19 12:00:00 2700371.8 301.42535 296.49295
2020-03-20 12:00:00 2572347.8 301.25507 297.3334
2020-03-21 12:00:00 2982751.0 301.481 297.96732
2020-03-22 12:00:00 2265461.8 300.53827 297.3621
2020-03-23 12:00:00 2992819.8 300.9136 297.10645
2020-03-

2020-04-02 12:00:00 2513158.5 300.8603 298.3934
2020-04-03 12:00:00 2925533.2 300.79013 298.3494
2020-04-04 12:00:00 2714452.8 301.3628 297.96033
2020-04-05 12:00:00 1704654.2 301.43427 298.09677
2020-04-06 12:00:00 2846674.0 301.51413 297.7748
2020-04-07 12:00:00 2739244.8 301.68823 297.57956
2020-04-08 12:00:00 2872384.0 301.40387 297.7986
2020-04-09 12:00:00 2693644.2 301.02173 297.85715
2020-04-10 12:00:00 2264796.8 301.29504 297.4727
2020-04-11 12:00:00 2840560.0 301.36276 297.54868
2020-04-12 12:00:00 2547679.0 301.04846 297.50955
2020-04-13 12:00:00 2793197.5 301.10974 298.11563
2020-04-14 12:00:00 2741234.5 301.33316 298.01236
2020-04-15 12:00:00 2823515.0 301.3563 298.329
2020-04-16 12:00:00 2726716.5 301.81088 298.11557
2020-04-17 12:00:00 2171390.8 301.3577 298.0063
2020-04-18 12:00:00 2827855.5 301.52026 297.70053
2020-04-19 12:00:00 2249983.8 300.23776 297.35498
2020-04-20 12:00:00 2376504.5 301.74292 297.56433
2020-04-21 12:00:00 2591225.2 301.42654 297.55276
Piaui
2020-0

In [45]:
#     if save:
max_bulk_create(msmt_objs)

/Users/spindicate/Documents/programming/envs/zooenv/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1368: RuntimeWarning: DateTimeField Measurements.date received a naive datetime (2020-01-01 12:00:00) while time zone support is active.
  RuntimeWarning)
/Users/spindicate/Documents/programming/envs/zooenv/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1368: RuntimeWarning: DateTimeField Measurements.date received a naive datetime (2020-01-02 12:00:00) while time zone support is active.
  RuntimeWarning)
/Users/spindicate/Documents/programming/envs/zooenv/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1368: RuntimeWarning: DateTimeField Measurements.date received a naive datetime (2020-01-03 12:00:00) while time zone support is active.
  RuntimeWarning)
/Users/spindicate/Documents/programming/envs/zooenv/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1368: RuntimeWarning: DateTimeField Measurements.date received a naive 

In [52]:
Measurements.objects.filter(region__name='Hong Kong').values()

<QuerySet [{'id': 315579, 'region_id': 353, 'date': datetime.datetime(2020, 1, 1, 12, 0, tzinfo=<UTC>), 'longitude': 114.2, 'latitude': 22.3, 'temp': 290.6557922363281, 'dewpoint': 287.1003723144531, 'uvb': 1518109.0, 'evap': None}, {'id': 315580, 'region_id': 353, 'date': datetime.datetime(2020, 1, 2, 12, 0, tzinfo=<UTC>), 'longitude': 114.2, 'latitude': 22.3, 'temp': 290.7848815917969, 'dewpoint': 287.8463439941406, 'uvb': 1432115.625, 'evap': None}, {'id': 315581, 'region_id': 353, 'date': datetime.datetime(2020, 1, 3, 12, 0, tzinfo=<UTC>), 'longitude': 114.2, 'latitude': 22.3, 'temp': 291.3793640136719, 'dewpoint': 288.7074279785156, 'uvb': 1696777.375, 'evap': None}, {'id': 315582, 'region_id': 353, 'date': datetime.datetime(2020, 1, 4, 12, 0, tzinfo=<UTC>), 'longitude': 114.2, 'latitude': 22.3, 'temp': 292.21630859375, 'dewpoint': 289.3677062988281, 'uvb': 1619373.25, 'evap': None}, {'id': 315583, 'region_id': 353, 'date': datetime.datetime(2020, 1, 5, 12, 0, tzinfo=<UTC>), 'long